<a target="_blank" href="https://colab.research.google.com/github/emiletimothy/Caltech-CS155-2023/blob/main/set6/HMM.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# CS155 Miniproject 3: Shakespearebot 5000 - Moles (Jesse, Adam, Andrea, Rolf)

**Imports**


In [2]:
import os
import re
import random
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from matplotlib import animation
from matplotlib.animation import FuncAnimation
import math
import string


def log(x):
    return math.log(x) if x > 0 else -math.inf

## Pre-processing
- Remove punctuation
- It's probably best to tokenize the words by their stresses (unstressed vs stressed) but that would require storing words as bigrams if they are over a certain length like 6 or smthn (splitting them down the middle)
- Sequence is a line? stanza (but then you have either the 4 line stanza or the 2 line stanza)? The poem itself? If we tokenize words by their stresses, then each sequence should probably be a line. 
- Don't split hyphenated words?


In [3]:
# process the syllable dictionary
with open('data/Syllable_dictionary.txt') as file:
    sdict = {}
    for line in file:
      data = line.split()
      key, value = data[0], data[1:]
      sdict[key] = value
  


In [83]:

def process_line(line):
    """asdf"""
    word_syllables_counts = []
    for word in line.split(" "):
        word = word.lower()
        try:
            counts = sdict[word]
        except KeyError:
            for punct in [",", ".", "!", "?", ":", "'", ";", "(", ")", "[", "]", "{", "}"]:
                word = word.replace(punct, "")
                if word in sdict:
                    break
            counts = sdict[word]
        word_syllables_counts.append((word, counts))
    syllables = process_counts(word_syllables_counts)
    return syllables

    
def process_counts(word_syllables_counts):
    def dfs(wsc, remaining_syllabes) :
        word, counts = wsc[0]
        if len(wsc) == 1:
            for count in counts:
                count = int(count.replace("E", ""))
                if count == remaining_syllabes:
                    return [[(word, count)]]
        else:
            valid_paths = []
            for count in counts:
                if "E" in count:
                    continue
                count = int(count)
                if count < remaining_syllabes:
                    paths = dfs(wsc[1:], remaining_syllabes - count)
                    for path in paths:
                        valid_paths.append([(word, count)] + path)
            return valid_paths
        return []

    valid_paths = dfs(word_syllables_counts, 10)

    if len(valid_paths) == 0:
        return None
    
    # choose first valid path
    path = valid_paths[0]

    return path

In [84]:
# load shakespeare data using pandas
import pandas as pd

shakespeare_url = "https://raw.githubusercontent.com/cs155-2020/lectures/main/lecture_1/shakespeare.txt"
shakespeare_file = "data/shakespeare.txt"
if not os.path.exists(shakespeare_file):
    urllib.request.urlretrieve(shakespeare_url, shakespeare_file)

shakespeare = pd.read_csv(shakespeare_file, sep="\t", header=None, names=["line"])
# remove all rows containing integers
shakespeare = shakespeare[~shakespeare.line.str.contains(r"\d")]

processed_lines = []
for index, row in shakespeare.iterrows():
    line = row["line"].strip()
    processed_line = process_line(line)
    if processed_line is not None:
        new_line = []
        prev_stress = "$"
        for word, count in processed_line:
            prefix = ""
            for i in range(count):
                prefix += prev_stress
                prev_stress = "/" if prev_stress == "$" else "$"
            new_line.append(prefix + word)
        processed_lines.append(new_line)

process_lines_integers = []
word_map = {}
word_map_inversed = {}
counter = 0
for line in processed_lines:
    new_line = []
    for word in line:
        if word not in word_map:
            word_map[counter] = word
            word_map_inversed[word] = counter
        new_line.append(counter)
        counter += 1
    process_lines_integers.append(new_line)

print(word_map)

['shifts', 'but', 'his', 'place', 'for', 'still', 'the', 'world', 'enjoys', 'it']
['and', 'kept', 'unused', 'the', 'user', 'so', 'destroys', 'it']
['may', 'make', 'seem', 'bare', 'in', 'wanting', 'words', 'to', 'show', 'it']
['in', 'thy', "soul's", 'thought', 'all', 'naked', 'will', 'bestow', 'it']
['then', 'may', 'i', 'dare', 'to', 'boast', 'how', 'i', 'do', 'love', 'thee']
['till', 'then', 'not', 'show', 'my', 'head', 'where', 'thou', 'mayst', 'prove', 'me']
['loving', 'offenders', 'thus', 'i', 'will', 'excuse', 'ye']
['thou', 'dost', 'love', 'her', 'because', 'thou', "know'st", 'i', 'love', 'her']
['and', 'for', 'my', 'sake', 'even', 'so', 'doth', 'she', 'abuse', 'me']
["suff'ring", 'my', 'friend', 'for', 'my', 'sake', 'to', 'approve', 'her']
['pity', 'me', 'then', 'dear', 'friend', 'and', 'i', 'assure', 'ye']
['even', 'that', 'your', 'pity', 'is', 'enough', 'to', 'cure', 'me']
['divert', 'strong', 'minds', 'to', 'the', 'course', 'of', "alt'ring", 'things']
['o', 'that', 'our', 'nig

## HMM CODE (TODO)


In [13]:
class HiddenMarkovModel:
    """
    Class implementation of Hidden Markov Models.
    """

    def __init__(self, A, O):
        """
        Initializes an HMM. Assumes the following:
            - States and observations are integers starting from 0.
            - There is a start state (see notes on A_start below). There
              is no integer associated with the start state, only
              probabilities in the vector A_start.
            - There is no end state.
        Arguments:
            A:          Transition matrix with dimensions L x L.
                        The (i, j)^th element is the probability of
                        transitioning from state i to state j. Note that
                        this does not include the starting probabilities.
            O:          Observation matrix with dimensions L x D.
                        The (i, j)^th element is the probability of
                        emitting observation j given state i.
        Parameters:
            L:          Number of states.

            D:          Number of observations.

            A:          The transition matrix.

            O:          The observation matrix.

            A_start:    Starting transition probabilities. The i^th element
                        is the probability of transitioning from the start
                        state to state i. For simplicity, we assume that
                        this distribution is uniform.
        """

        self.L = len(A)
        self.D = len(O[0])
        self.A = A
        self.O = O
        self.A_start = [1.0 / self.L for _ in range(self.L)]

    def viterbi(self, x):
        """
        Uses the Viterbi algorithm to find the max probability state
        sequence corresponding to a given input sequence.
        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.
        Returns:
            max_seq:    State sequence corresponding to x with the highest
                        probability.
        """

        M = len(x)  # Length of sequence.

        # The (i, j)^th elements of probs and seqs are the max probability
        # of the prefix of length i ending in state j and the prefix
        # that gives this probability, respectively.
        #
        # For instance, probs[1][0] is the probability of the prefix of
        # length 1 ending in state 0.
        #
        # probs stores log probabilities!!!
        probs = [[0.0 for _ in range(self.L)] for _ in range(M + 1)]
        seqs = [["" for _ in range(self.L)] for _ in range(M + 1)]

        probs[1] = [
            log(self.A_start[state] * self.O[state][x[0]]) for state in range(self.L)
        ]
        seqs[1] = [str(state) for state in range(self.L)]

        for length in range(2, M + 1):
            for state in range(self.L):
                max_prob = -math.inf
                max_seq = None
                for prev_state in range(self.L):
                    prob = (
                        probs[length - 1][prev_state]
                        + log(self.A[prev_state][state])
                        + log(self.O[state][x[length - 1]])
                    )

                    if prob > max_prob:
                        max_prob = prob
                        max_seq = seqs[length - 1][prev_state] + str(state)

                probs[length][state] = max_prob
                seqs[length][state] = max_seq

        max_seq = max(
            seqs[M],
            key=lambda seq: probs[M][int(seq[-1])] if seq is not None else -math.inf,
        )

        return max_seq

    def forward(self, x, normalize=False):
        """
        Uses the forward algorithm to calculate the alpha probability
        vectors corresponding to a given input sequence.
        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.
            normalize:  Whether to normalize each set of alpha_j(i) vectors
                        at each i. This is useful to avoid underflow in
                        unsupervised learning.
        Returns:
            alphas:     Vector of alphas.
                        The (i, j)^th element of alphas is alpha_j(i),
                        i.e. the probability of observing prefix x^1:i
                        and state y^i = j.
                        e.g. alphas[1][0] corresponds to the probability
                        of observing x^1:1, i.e. the first observation,
                        given that y^1 = 0, i.e. the first state is 0.
        """

        M = len(x)  # Length of sequence.
        alphas = [[0.0 for _ in range(self.L)] for _ in range(M + 1)]

        alphas[1] = [
            self.A_start[state] * self.O[state][x[0]] for state in range(self.L)
        ]

        for length in range(2, M + 1):
            for state in range(self.L):
                alphas[length][state] = self.O[state][x[length - 1]] * sum(
                    [
                        alphas[length - 1][prev_state] * self.A[prev_state][state]
                        for prev_state in range(self.L)
                    ]
                )
            if normalize:
                normalization_constant = sum(alphas[length])
                for state in range(self.L):
                    alphas[length][state] /= normalization_constant

        return alphas

    def backward(self, x, normalize=False):
        """
        Uses the backward algorithm to calculate the beta probability
        vectors corresponding to a given input sequence.
        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.
            normalize:  Whether to normalize each set of alpha_j(i) vectors
                        at each i. This is useful to avoid underflow in
                        unsupervised learning.
        Returns:
            betas:      Vector of betas.
                        The (i, j)^th element of betas is beta_j(i), i.e.
                        the probability of observing prefix x^(i+1):M and
                        state y^i = j.
                        e.g. betas[M][0] corresponds to the probability
                        of observing x^M+1:M, i.e. no observations,
                        given that y^M = 0, i.e. the last state is 0.
        """

        M = len(x)  # Length of sequence.
        betas = [[0.0 for _ in range(self.L)] for _ in range(M + 1)]

        betas[M] = [1.0 for _ in range(self.L)]
        for length in range(M - 1, -1, -1):
            for state in range(self.L):
                betas[length][state] = sum(
                    [
                        betas[length + 1][next_state]
                        * self.A[state][next_state]
                        * self.O[next_state][x[length]]
                        for next_state in range(self.L)
                    ]
                )
            if normalize:
                normalization_constant = sum(betas[length])
                for state in range(self.L):
                    betas[length][state] /= normalization_constant

        return betas

    def supervised_learning(self, X, Y):
        """
        Trains the HMM using the Maximum Likelihood closed form solutions
        for the transition and observation matrices on a labeled
        datset (X, Y). Note that this method does not return anything, but
        instead updates the attributes of the HMM object.
        Arguments:
            X:          A dataset consisting of input sequences in the form
                        of lists of variable length, consisting of integers
                        ranging from 0 to D - 1. In other words, a list of
                        lists.
            Y:          A dataset consisting of state sequences in the form
                        of lists of variable length, consisting of integers
                        ranging from 0 to L - 1. In other words, a list of
                        lists.
                        Note that the elements in X line up with those in Y.
        """
        N = len(X)

        # Calculate each element of A using the M-step formulas.

        self.A = [[0.0 for _ in range(self.L)] for _ in range(self.L)]

        for a in range(self.L):
            for b in range(self.L):
                nominator = 0
                denominator = 0
                for i in range(N):
                    for k in range(1, len(X[i])):
                        if Y[i][k - 1] == a:
                            denominator += 1
                            if Y[i][k] == b:
                                nominator += 1
                self.A[a][b] = nominator / denominator

        # Calculate each element of O using the M-step formulas.

        self.O = [[0.0 for _ in range(self.D)] for _ in range(self.L)]

        for a in range(self.L):
            for w in range(self.D):
                nominator = 0
                denominator = 0
                for i in range(N):
                    for k in range(0, len(X[i])):
                        if Y[i][k] == a:
                            denominator += 1
                            if X[i][k] == w:
                                nominator += 1
                self.O[a][w] = nominator / denominator

    def unsupervised_learning(self, X, N_iters):
        """
        Trains the HMM using the Baum-Welch algorithm on an unlabeled
        datset X. Note that this method does not return anything, but
        instead updates the attributes of the HMM object.
        Arguments:
            X:          A dataset consisting of input sequences in the form
                        of variable-length lists, consisting of integers
                        ranging from 0 to D - 1. In other words, a list of
                        lists.
            N_iters:    The number of iterations to train on.
        """
        N = len(X)

        for iter in range(N_iters):
            print("iter:", iter, end="\r")
            new_A = [[0.0 for b in range(self.L)] for a in range(self.L)]
            new_O = [[0.0 for w in range(self.D)] for a in range(self.L)]

            for i in range(N):
                alphas = self.forward(X[i], normalize=True)
                betas = self.backward(X[i], normalize=True)

                for k in range(len(X[i])):
                    add_O = [alphas[k + 1][a] * betas[k + 1][a] for a in range(self.L)]
                    denom = sum(add_O)
                    for a in range(self.L):
                        for w in range(self.D):
                            if X[i][k] == w:
                                new_O[a][w] += add_O[a] / denom

                    if k != 0:
                        add_A = [
                            [
                                alphas[k][a]
                                * self.O[b][X[i][k]]
                                * self.A[a][b]
                                * betas[k + 1][b]
                                for b in range(self.L)
                            ]
                            for a in range(self.L)
                        ]

                        denom = 0
                        for a in range(self.L):
                            for b in range(self.L):
                                denom += add_A[a][b]

                        for a in range(self.L):
                            for b in range(self.L):
                                new_A[a][b] += add_A[a][b] / denom

            # normalizing A
            for a in range(self.L):
                denom = sum(new_A[a])
                for b in range(self.L):
                    new_A[a][b] /= denom

            # normalizing O
            for a in range(self.L):
                denom = sum(new_O[a])
                for w in range(self.D):
                    new_O[a][w] /= denom

            # updating A and O
            self.A = new_A
            self.O = new_O

    def generate_emission(self, M, end_words=[], seed=None):
        """
        Generates an emission of length M, assuming that the starting state
        is chosen uniformly at random.
        Arguments:
            M:          Length of the emission to generate.
        Returns:
            emission:   The randomly generated emission as a list.
            states:     The randomly generated states as a list.
        """

        # transpose self.A
        A_trans = list(zip(*self.A))
        O_trans = list(zip(*self.O))

        A_start = [0.0 for _ in range(self.L)]
        for word in end_words:
            p = O_trans[word]
            for i in range(self.L):
                A_start[i] += p[i]
        A_start = (
            [x / sum(A_start) for x in A_start]
            if sum(A_start) != 0
            else [1 / self.L for _ in range(self.L)]
        )

        # (Re-)Initialize random number generator
        rng = np.random.default_rng(seed=seed)

        emission = []
        states = []

        syllables = 0

        def get_syllable_count(word):
            syllable_c = 0
            for char in word:
                if char == "$" or char == "/":
                    syllable_c += 1
            return syllable_c

        while syllables < M:
            new_state = rng.choice(
                range(self.L),
                p=[p / sum(A_trans[states[-1]]) for p in A_trans[states[-1]]]
                if len(states) > 0
                else A_start,
            )
            states.append(new_state)

            if len(states) == 1:
                seq = rng.choice(end_words)
            else:
                seq = rng.choice(range(self.D), p=self.O[new_state])
            word = word_map[seq]
            syllable_c = get_syllable_count(word)

            while syllable_c + syllables > 10:
                seq = rng.choice(range(self.D), p=self.O[new_state])
                word = word_map[seq]
                syllable_c = get_syllable_count(word)

            syllables += syllable_c
            emission.append(seq)

        return list(reversed(emission)), list(reversed(states))

    def probability_alphas(self, x):
        """
        Finds the maximum probability of a given input sequence using
        the forward algorithm.
        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.
        Returns:
            prob:       Total probability that x can occur.
        """

        # Calculate alpha vectors.
        alphas = self.forward(x)

        # alpha_j(M) gives the probability that the state sequence ends
        # in j. Summing this value over all possible states j gives the
        # total probability of x paired with any state sequence, i.e.
        # the probability of x.
        prob = sum(alphas[-1])
        return prob

    def probability_betas(self, x):
        """
        Finds the maximum probability of a given input sequence using
        the backward algorithm.
        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.
        Returns:
            prob:       Total probability that x can occur.
        """

        betas = self.backward(x)

        # beta_j(1) gives the probability that the state sequence starts
        # with j. Summing this, multiplied by the starting transition
        # probability and the observation probability, over all states
        # gives the total probability of x paired with any state
        # sequence, i.e. the probability of x.
        prob = sum(
            [betas[1][j] * self.A_start[j] * self.O[j][x[0]] for j in range(self.L)]
        )

        return prob


def supervised_HMM(X, Y):
    """
    Helper function to train a supervised HMM. The function determines the
    number of unique states and observations in the given data, initializes
    the transition and observation matrices, creates the HMM, and then runs
    the training function for supervised learning.
    Arguments:
        X:          A dataset consisting of input sequences in the form
                    of lists of variable length, consisting of integers
                    ranging from 0 to D - 1. In other words, a list of lists.
        Y:          A dataset consisting of state sequences in the form
                    of lists of variable length, consisting of integers
                    ranging from 0 to L - 1. In other words, a list of lists.
                    Note that the elements in X line up with those in Y.
    """
    # Make a set of observations.
    observations = set()
    for x in X:
        observations |= set(x)

    # Make a set of states.
    states = set()
    for y in Y:
        states |= set(y)

    # Compute L and D.
    L = len(states)
    D = len(observations)

    # Randomly initialize and normalize matrix A.
    A = [[random.random() for i in range(L)] for j in range(L)]

    for i in range(len(A)):
        norm = sum(A[i])
        for j in range(len(A[i])):
            A[i][j] /= norm

    # Randomly initialize and normalize matrix O.
    O = [[random.random() for i in range(D)] for j in range(L)]

    for i in range(len(O)):
        norm = sum(O[i])
        for j in range(len(O[i])):
            O[i][j] /= norm

    # Train an HMM with labeled data.
    HMM = HiddenMarkovModel(A, O)
    HMM.supervised_learning(X, Y)

    return HMM


def unsupervised_HMM(X, n_states, N_iters, seed=None):
    """
    Helper function to train an unsupervised HMM. The function determines the
    number of unique observations in the given data, initializes
    the transition and observation matrices, creates the HMM, and then runs
    the training function for unsupervised learing.
    Arguments:
        X:          A dataset consisting of input sequences in the form
                    of lists of variable length, consisting of integers
                    ranging from 0 to D - 1. In other words, a list of lists.
        n_states:   Number of hidden states to use in training.

        N_iters:    The number of iterations to train on.
        rng:        The random number generator for reproducible result.
                    Default to RandomState(1).
    """
    # Initialize random number generator
    rng = np.random.default_rng(seed=seed)

    # Make a set of observations.
    observations = set()
    for x in X:
        observations |= set(x)

    # Compute L and D.
    L = n_states
    D = len(observations)

    # Randomly initialize and normalize matrix A.
    A = [[rng.random() for i in range(L)] for j in range(L)]

    for i in range(len(A)):
        norm = sum(A[i])
        for j in range(len(A[i])):
            A[i][j] /= norm

    # Randomly initialize and normalize matrix O.
    O = [[rng.random() for i in range(D)] for j in range(L)]

    for i in range(len(O)):
        norm = sum(O[i])
        for j in range(len(O[i])):
            O[i][j] /= norm

    # Train an HMM with unlabeled data.
    HMM = HiddenMarkovModel(A, O)
    HMM.unsupervised_learning(X, N_iters)

    return HMM

**HMM helper code. No need to modify anything here.**


In [7]:
########################################
# CS/CNS/EE 155 2018
# Problem Set 6
#
# Author:       Andrew Kang
# Description:  Set 6 HMM helper
########################################

import re
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from matplotlib import animation
from matplotlib.animation import FuncAnimation


####################
# WORDCLOUD FUNCTIONS
####################


def mask():
    # Parameters.
    r = 128
    d = 2 * r + 1

    # Get points in a circle.
    y, x = np.ogrid[-r: d - r, -r: d - r]
    circle = x**2 + y**2 <= r**2

    # Create mask.
    mask = 255 * np.ones((d, d), dtype=np.uint8)
    mask[circle] = 0

    return mask


def text_to_wordcloud(text, max_words=50, title="", show=True):
    plt.close("all")

    # Generate a wordcloud image.
    wordcloud = WordCloud(
        random_state=0, max_words=max_words, background_color="white", mask=mask()
    ).generate(text)

    # Show the image.
    if show:
        plt.imshow(wordcloud, interpolation="bilinear")
        plt.axis("off")
        plt.title(title, fontsize=24)
        plt.show()

    return wordcloud


def states_to_wordclouds(hmm, obs_map, max_words=50, show=True):
    # Initialize.
    M = 100000
    n_states = len(hmm.A)
    obs_map_r = obs_map_reverser(obs_map)
    wordclouds = []

    # Generate a large emission.
    emission, states = hmm.generate_emission(M)

    # For each state, get a list of observations that have been emitted
    # from that state.
    obs_count = []
    for i in range(n_states):
        obs_lst = np.array(emission)[np.where(np.array(states) == i)[0]]
        obs_count.append(obs_lst)

    # For each state, convert it into a wordcloud.
    for i in range(n_states):
        obs_lst = obs_count[i]
        sentence = [obs_map_r[j] for j in obs_lst]
        sentence_str = " ".join(sentence)

        wordclouds.append(
            text_to_wordcloud(
                sentence_str, max_words=max_words, title="State %d" % i, show=show
            )
        )

    return wordclouds


####################
# HMM FUNCTIONS
####################


def parse_observations(text):
    # Convert text to dataset.
    lines = [line.split() for line in text.split("\n") if line.split()]

    obs_counter = 0
    obs = []
    obs_map = {}

    for line in lines:
        obs_elem = []

        for word in line:
            word = re.sub(r"[^\w]", "", word).lower()
            if word not in obs_map:
                # Add unique words to the observations map.
                obs_map[word] = obs_counter
                obs_counter += 1

            # Add the encoded word.
            obs_elem.append(obs_map[word])

        # Add the encoded sequence.
        obs.append(obs_elem)

    return obs, obs_map


def obs_map_reverser(obs_map):
    obs_map_r = {}

    for key in obs_map:
        obs_map_r[obs_map[key]] = key

    return obs_map_r


def sample_sentence(hmm, obs_map, n_words=100, seed=None):
    # Get reverse map.
    obs_map_r = obs_map_reverser(obs_map)

    # Sample and convert sentence.
    emission, states = hmm.generate_emission(n_words, seed=seed)
    sentence = [obs_map_r[i] for i in emission]

    return " ".join(sentence).capitalize() + "..."


####################
# HMM VISUALIZATION FUNCTIONS
####################


def visualize_sparsities(hmm, O_max_cols=50, O_vmax=0.1):
    plt.close("all")
    plt.set_cmap("viridis")

    # Visualize sparsity of A.
    plt.imshow(hmm.A, vmax=1.0)
    plt.colorbar()
    plt.title("Sparsity of A matrix")
    plt.show()

    # Visualize parsity of O.
    plt.imshow(np.array(hmm.O)[:, :O_max_cols], vmax=O_vmax, aspect="auto")
    plt.colorbar()
    plt.title("Sparsity of O matrix")
    plt.show()


####################
# HMM ANIMATION FUNCTIONS
####################


def animate_emission(hmm, obs_map, M=8, height=12, width=12, delay=1, seed=None):
    # Parameters.
    lim = 1200
    text_x_offset = 40
    text_y_offset = 80
    x_offset = 580
    y_offset = 520
    R = 420
    r = 100
    arrow_size = 20
    arrow_p1 = 0.03
    arrow_p2 = 0.02
    arrow_p3 = 0.06

    # Initialize.
    n_states = len(hmm.A)
    obs_map_r = obs_map_reverser(obs_map)
    wordclouds = states_to_wordclouds(hmm, obs_map, max_words=20, show=False)

    # Initialize plot.
    fig, ax = plt.subplots()
    fig.set_figheight(height)
    fig.set_figwidth(width)
    ax.grid("off")
    plt.axis("off")
    ax.set_xlim([0, lim])
    ax.set_ylim([0, lim])

    # Plot each wordcloud.
    for i, wordcloud in enumerate(wordclouds):
        x = x_offset + int(R * np.cos(np.pi * 2 * i / n_states))
        y = y_offset + int(R * np.sin(np.pi * 2 * i / n_states))
        ax.imshow(
            wordcloud.to_array(),
            extent=(x - r, x + r, y - r, y + r),
            aspect="auto",
            zorder=-1,
        )

    # Initialize text.
    text = ax.text(text_x_offset, lim - text_y_offset, "", fontsize=24)

    # Make the arrows.
    zorder_mult = n_states**2 * 100
    arrows = []
    for i in range(n_states):
        row = []
        for j in range(n_states):
            # Arrow coordinates.
            x_i = x_offset + R * np.cos(np.pi * 2 * i / n_states)
            y_i = y_offset + R * np.sin(np.pi * 2 * i / n_states)
            x_j = x_offset + R * np.cos(np.pi * 2 * j / n_states)
            y_j = y_offset + R * np.sin(np.pi * 2 * j / n_states)

            dx = x_j - x_i
            dy = y_j - y_i
            d = np.sqrt(dx**2 + dy**2)

            if i != j:
                arrow = ax.arrow(
                    x_i + (r / d + arrow_p1) * dx + arrow_p2 * dy,
                    y_i + (r / d + arrow_p1) * dy + arrow_p2 * dx,
                    (1 - 2 * r / d - arrow_p3) * dx,
                    (1 - 2 * r / d - arrow_p3) * dy,
                    color=(1 - hmm.A[i][j],) * 3,
                    head_width=arrow_size,
                    head_length=arrow_size,
                    zorder=int(hmm.A[i][j] * zorder_mult),
                )
            else:
                arrow = ax.arrow(
                    x_i,
                    y_i,
                    0,
                    0,
                    color=(1 - hmm.A[i][j],) * 3,
                    head_width=arrow_size,
                    head_length=arrow_size,
                    zorder=int(hmm.A[i][j] * zorder_mult),
                )

            row.append(arrow)
        arrows.append(row)

    emission, states = hmm.generate_emission(M, seed=seed)

    def animate(i):
        if i >= delay:
            i -= delay

            if i == 0:
                arrows[states[0]][states[0]].set_color("red")
            elif i == 1:
                arrows[states[0]][states[0]].set_color(
                    (1 - hmm.A[states[0]][states[0]],) * 3
                )
                arrows[states[i - 1]][states[i]].set_color("red")
            else:
                arrows[states[i - 2]][states[i - 1]].set_color(
                    (1 - hmm.A[states[i - 2]][states[i - 1]],) * 3
                )
                arrows[states[i - 1]][states[i]].set_color("red")

            # Set text.
            text.set_text(
                " ".join([obs_map_r[e]
                         for e in emission][: i + 1]).capitalize()
            )

            return arrows + [text]

    # Animate!
    print("\nAnimating...")
    anim = FuncAnimation(fig, animate, frames=M + delay, interval=1000)

    return anim


**Additional helper code. No need to modify anything here**


## Rhymes

In [55]:
def get_rhyme(filename):
  '''
  Returns dictionary with key as initial word and value as its rhymes
  '''
  with open(filename) as file:
    rdict = {}
    poemdict = {}
    new_lwl = []
    last_words_list = []
    curr = None
    for line in file:
      if any(char.isdigit() for char in line) or not line.strip():
            poemdict[line] = []
            curr = line
      else:
        poemdict[curr].append(line)
    
    for poem in poemdict:
      tmp = []
      for line in poemdict[poem]:
        words = line.split()
        last_word = words[-1].strip('.,?!;:)')
        tmp.append(last_word)
      new_lwl.append(tmp)
    

    for last_words_list in new_lwl:
      if len(last_words_list) == 14:
        for i, key in enumerate(last_words_list):
          if i == 0 or i == 1 or i == 4 or i == 5 or i == 8 or i == 9: 
              first =  last_words_list[i] + " "+str(i)
              second = last_words_list[i+2] + " "+str(i)
              rdict[first] = second
              rdict[second] = first
          if i == 12:
              first =  last_words_list[i] + " "+str(i)
              second = last_words_list[i+1] + " "+str(i)
              rdict[first] = second
              rdict[second] = first
    

    rdictcombined = {}
    for each in rdict:
      split = each.split()
      newkey = split[0].lower()
      split = rdict[each].split()
      newval = split[0].lower()

      if newkey not in rdictcombined:
        rdictcombined[newkey] = set([])
      rdictcombined[newkey].add(newval)

    return rdictcombined

In [57]:
rhymes = get_rhyme('data/shakespeare.txt')

## Unsupervised Learning

In [17]:
# Train the HMM.
HMM = unsupervised_HMM(process_lines_integers, 4, 3)

In [75]:
def prefix_word(word):
    if word in sdict:
        counts = sdict[word]
    else:
        for punct in [",", ".", "!", "?", ":", "'", ";", "(", ")", "[", "]", "{", "}"]:
            word = word.replace(punct, "")
            if word in sdict:
                break
        counts = sdict[word]
    
    end_counts = [c for c in counts if c[0] == "E"]
    count = end_counts[0] if len(end_counts) > 0 else counts[0]

    stress = "/"
    prefix = ""
    for _ in range(int(count.replace("E", ""))):
        prefix = stress + prefix 
        stress = "/" if stress == "$" else "$"
    
    return prefix + word

In [85]:
def print_emission(emission, remove_special=False):
    if remove_special:
        print(" ".join([word_map[id].replace("$", "").replace("/", "") for id in emission]))
    else:
        print(" ".join([word_map[id] for id in emission]))

def invert_words(words):
    return [word_map_inversed[prefix_word(word)] for word in words if prefix_word(word) in word_map_inversed]

In [80]:
"ye" in rhymes.keys()

True

In [86]:
line_pairs = []
for _ in range(7):
    emission1, _ = HMM.generate_emission(10, end_words=invert_words(rhymes.keys()))
    last_word = word_map[emission1[-1]].replace("$", "").replace("/", "")
    emission2, _ = HMM.generate_emission(10, end_words=invert_words(rhymes[last_word]))

    line_pairs.append((emission1, emission2))

In [87]:
poem = [
    [
        line_pairs[0][0],  # a1
        line_pairs[1][0],  # b1
        line_pairs[0][1],  # a2
        line_pairs[1][1],  # b2
    ],  
    [
        line_pairs[2][0],  # c1
        line_pairs[3][0],  # d1
        line_pairs[2][1],  # c2
        line_pairs[3][1],  # d2
    ],  
    [
        line_pairs[4][0],  # e1
        line_pairs[5][0],  # f1
        line_pairs[4][1],  # e2
        line_pairs[5][1],  # f2
    ],
    [line_pairs[6][0], line_pairs[6][1]],  # g1  # g2
]

In [89]:
for stanza in poem:
    for line in stanza:
        print_emission(line, remove_special=True)
    print()

/catch $/stealing /out /name $in $/$/society
/end $with $as $on $/triumph $nor $an $and /owe
/to /sky $by /one /gates /of $/$/impiety
/you $hung /hadst $/enough /back /$virtue /words /show

/$/excellence $grow $the $on $/alone $could /kind
/thou $/$enjoyer $keep /fair /part /$every /haste
/will $a /those /$wrinkle $how /$bosom's /to /mind
/i /world $wilt /leave $too $as /$/covetous /past

/$gentle /that $that /$/ornament $that $on /scope
$and $am $fierce $hath $false /make $drawn $as $/confined
$have /now $the $of $were $most $/enough /to /hope
/and $what /ground $of /with /know /the $the /that /kind

$till $though /me /stand $and /thee $death's $so $that /rage
/smell /$crystal $till /his /space /bear $why /for /age

